# Classify the tiles using pretrained tile2vec
In this notebook we are going to measure the performance of classifier on EuroSATallbands dataset.

In [1]:
import numpy as np
import os
import torch
from time import time
from torch.autograd import Variable
import random


import sys
sys.path.append('../')
from src.tilenet import make_tilenet
from src.resnet import ResNet18
import pandas as pd
from pathlib import Path
from osgeo import gdal

# sample tiles

In [2]:
sample_tiles_train_path = "/storage/EuroSATallBands/train.csv"
train_paths = pd.read_csv(sample_tiles_train_path)

base_eurosat_dir = "/storage/EuroSATallBands"

train_paths

,Filename,Label,ClassName
0,PermanentCrop/PermanentCrop_2401.tif,6,PermanentCrop
1,PermanentCrop/PermanentCrop_1006.tif,6,PermanentCrop
2,HerbaceousVegetation/HerbaceousVegetation_1025...,2,HerbaceousVegetation
3,SeaLake/SeaLake_1439.tif,9,SeaLake
4,River/River_1052.tif,8,River
...,...,...,...
19312,HerbaceousVegetation/HerbaceousVegetation_2292...,2,HerbaceousVegetation
19313,AnnualCrop/AnnualCrop_1226.tif,0,AnnualCrop
19314,SeaLake/SeaLake_2010.tif,9,SeaLake
19315,SeaLake/SeaLake_2291.tif,9,SeaLake


In [3]:
# create tiles

def get_triplet_imgs(img_df, n_triplets=1000):
    """
    Returns a numpy array of dimension (n_triplets, 2). First column is
    the img name of anchor/neighbor tiles and second column is img name 
    of distant tiles.
    """
    img_names = []
    for filename in img_df["Filename"]:
        img_names.append(filename)
    img_triplets = list(map(lambda _: random.choice(img_names), range(2 * n_triplets)))
    img_triplets = np.array(img_triplets)
    return img_triplets.reshape((-1, 2))


img_triplets = get_triplet_imgs(train_paths)
img_triplets

array([['Forest/Forest_2383.tif', 'Industrial/Industrial_2201.tif'],
       ['Forest/Forest_1940.tif', 'Industrial/Industrial_630.tif'],
       ['SeaLake/Jakarta_000437.tif', 'River/River_1058.tif'],
       ...,
       ['Pasture/Pasture_427.tif',
        'PermanentCrop/PermanentCrop_1676.tif'],
       ['Forest/Forest_1476.tif', 'Residential/Residential_1383.tif'],
       ['SeaLake/SeaLake_2494.tif', 'Residential/Residential_2192.tif']],
      dtype='<U50')

In [3]:
from src.sample_tiles import extract_tile, sample_distant_same, sample_neighbor, load_img, sample_anchor, sample_distant_diff


In [4]:

def get_triplet_tiles(tile_dir, img_dir, img_triplets, tile_size=50, neighborhood=100, 
                      val_type='uint8', bands_only=False, save=True, verbose=False):
    if not os.path.exists(tile_dir):
        os.makedirs(tile_dir)
    size_even = (tile_size % 2 == 0)
    tile_radius = tile_size // 2

    n_triplets = img_triplets.shape[0]
    unique_imgs = np.unique(img_triplets)
    tiles = np.zeros((n_triplets, 3, 2), dtype=np.int16)

    for img_name in unique_imgs:
        print("Sampling image {} from dir".format(img_name))
        if img_name[-3:] == 'npy':
            img = np.load(os.path.join(img_dir, img_name))
        else:
            img = load_img(os.path.join(img_dir, img_name), val_type=val_type, 
                       bands_only=bands_only)
        img_padded = np.pad(img, pad_width=[(tile_radius, tile_radius),
                                            (tile_radius, tile_radius), (0,0)],
                            mode='reflect')
        img_shape = img_padded.shape

        for idx, row in enumerate(img_triplets):
            if row[0] == img_name:
                xa, ya = sample_anchor(img_shape, tile_radius)
                xn, yn = sample_neighbor(img_shape, xa, ya, neighborhood, tile_radius)
                
                if verbose:
                    print("    Saving anchor and neighbor tile #{}".format(idx))
                    print("    Anchor tile center:{}".format((xa, ya)))
                    print("    Neighbor tile center:{}".format((xn, yn)))
                if save:
                    tile_anchor = extract_tile(img_padded, xa, ya, tile_radius)
                    tile_neighbor = extract_tile(img_padded, xn, yn, tile_radius)
                    if size_even:
                        tile_anchor = tile_anchor[:-1,:-1]
                        tile_neighbor = tile_neighbor[:-1,:-1]
                    np.save(os.path.join(tile_dir, '{}anchor.npy'.format(idx)), tile_anchor)
                    np.save(os.path.join(tile_dir, '{}neighbor.npy'.format(idx)), tile_neighbor)
                
                tiles[idx,0,:] = xa - tile_radius, ya - tile_radius
                tiles[idx,1,:] = xn - tile_radius, yn - tile_radius
                
                if row[1] == img_name:
                    # distant image is same as anchor/neighbor image
                    xd, yd = sample_distant_same(img_shape, xa, ya, neighborhood, tile_radius)
                    if verbose:
                        print("    Saving distant tile #{}".format(idx))
                        print("    Distant tile center:{}".format((xd, yd)))
                    if save:
                        tile_distant = extract_tile(img_padded, xd, yd, tile_radius)
                        if size_even:
                            tile_distant = tile_distant[:-1,:-1]
                        np.save(os.path.join(tile_dir, '{}distant.npy'.format(idx)), tile_distant)
                    tiles[idx,2,:] = xd - tile_radius, yd - tile_radius
            
            elif row[1] == img_name: 
                # distant image is different from anchor/neighbor image
                xd, yd = sample_distant_diff(img_shape, tile_radius)
                if verbose:
                        print("    Saving distant tile #{}".format(idx))
                        print("    Distant tile center:{}".format((xd, yd)))
                if save:
                    tile_distant = extract_tile(img_padded, xd, yd, tile_radius)
                    if size_even:
                        tile_distant = tile_distant[:-1,:-1]
                    np.save(os.path.join(tile_dir, '{}distant.npy'.format(idx)), tile_distant)
                tiles[idx,2,:] = xd - tile_radius, yd - tile_radius
            
    return tiles

In [5]:
# save the triplets
tile_dir = "/storage/tile2vec/tiles"

tiles = get_triplet_tiles(tile_dir, base_eurosat_dir, img_triplets)

Sampling image AnnualCrop/AnnualCrop_1013.tif from dir
Sampling image AnnualCrop/AnnualCrop_105.tif from dir
Sampling image AnnualCrop/AnnualCrop_1058.tif from dir
Sampling image AnnualCrop/AnnualCrop_108.tif from dir
Sampling image AnnualCrop/AnnualCrop_1081.tif from dir
Sampling image AnnualCrop/AnnualCrop_1103.tif from dir
Sampling image AnnualCrop/AnnualCrop_1106.tif from dir
Sampling image AnnualCrop/AnnualCrop_1129.tif from dir
Sampling image AnnualCrop/AnnualCrop_1142.tif from dir
Sampling image AnnualCrop/AnnualCrop_1178.tif from dir
Sampling image AnnualCrop/AnnualCrop_1200.tif from dir
Sampling image AnnualCrop/AnnualCrop_1204.tif from dir
Sampling image AnnualCrop/AnnualCrop_1209.tif from dir
Sampling image AnnualCrop/AnnualCrop_1218.tif from dir
Sampling image AnnualCrop/AnnualCrop_1236.tif from dir
Sampling image AnnualCrop/AnnualCrop_1239.tif from dir
Sampling image AnnualCrop/AnnualCrop_1244.tif from dir
Sampling image AnnualCrop/AnnualCrop_1274.tif from dir
Sampling ima

/home/tymek/.conda/envs/tile2vec/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Sampling image AnnualCrop/AnnualCrop_1576.tif from dir
Sampling image AnnualCrop/AnnualCrop_1582.tif from dir
Sampling image AnnualCrop/AnnualCrop_1601.tif from dir
Sampling image AnnualCrop/AnnualCrop_1613.tif from dir
Sampling image AnnualCrop/AnnualCrop_1620.tif from dir
Sampling image AnnualCrop/AnnualCrop_1623.tif from dir
Sampling image AnnualCrop/AnnualCrop_1641.tif from dir
Sampling image AnnualCrop/AnnualCrop_166.tif from dir
Sampling image AnnualCrop/AnnualCrop_1664.tif from dir
Sampling image AnnualCrop/AnnualCrop_1669.tif from dir
Sampling image AnnualCrop/AnnualCrop_1674.tif from dir
Sampling image AnnualCrop/AnnualCrop_1688.tif from dir
Sampling image AnnualCrop/AnnualCrop_1692.tif from dir
Sampling image AnnualCrop/AnnualCrop_1748.tif from dir
Sampling image AnnualCrop/AnnualCrop_1776.tif from dir
Sampling image AnnualCrop/AnnualCrop_1823.tif from dir
Sampling image AnnualCrop/AnnualCrop_1847.tif from dir
Sampling image AnnualCrop/AnnualCrop_1905.tif from dir
Sampling im

## train the new model

In [4]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet
from src.training import prep_triplets, train_triplet_epoch
from torch import optim


In [5]:
# Setting up model
in_channels = 13
z_dim = 512
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()
cuda

True

In [6]:
img_type = "naip" #images are in float - this parameter specifies that there is a need for normalization of floats
tile_dir = '/storage/tile2vec/tiles'
bands = 13
augment = True
batch_size = 50
shuffle = True
num_workers = 4
n_triplets = 1000

In [7]:
dataloader = triplet_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


In [8]:
in_channels = bands
z_dim = 512

In [9]:
TileNet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
TileNet.train()
if cuda: TileNet.cuda()
print('TileNet set up complete.')

TileNet set up complete.


In [10]:
cuda

True

In [11]:
lr = 1e-3
optimizer = optim.Adam(TileNet.parameters(), lr=lr, betas=(0.5, 0.999))

## training

In [12]:
epochs = 1
margin = 10
l2 = 0.01
print_every = 100
save_models = False

In [13]:
model_dir = '/storage/tile2vec/models'
if not os.path.exists(model_dir): 
    os.makedirs(model_dir)

In [14]:
results_fn = "/storage/tile2vec/results_fn"

In [15]:
t0 = time()
with open(results_fn, 'w') as file:

    print('Begin training.................')
    for epoch in range(0, epochs):
        (avg_loss, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
            TileNet, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
            print_every=print_every, t0=t0)

Begin training.................


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [27]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)
acc = rf.score(X_te, y_te)
print(f"Model accuracy: {acc}")

Model accuracy: 0.6868530020703933


not bad